In [2]:
import polars as pl

In [ ]:
# load data

# Food stamps (SNAP), Census Tract (2019-2023)
# source: Chicago Health Atlas https://chicagohealthatlas.org/download
snap = pl.read_csv("data/snap_data.csv", 
                            ignore_errors=True,
                            schema_overrides={"GEOID": pl.Utf8})

# Census Tract to Community Area Equivalency File 
# source: https://guides.lib.uchicago.edu/c.php?g=720045&p=8072546
census_crosswalk = pl.read_csv("data/census_tract_crosswalk.csv", 
                            ignore_errors=True,
                            schema_overrides={"Tract": pl.Utf8})

In [4]:
# extract geocode, round values

snap_df = snap.with_columns(
    pl.col("GEOID").str.slice(2).alias("GEOCODE"), 
    pl.col("INC_2019-2023").round(0),
    pl.col("PCT-W_2019-2023").round(1),
    pl.col("SNP_2019-2023").round(1)
)

In [ ]:
# merge snap data with census crosswalk to match neighborhood name with snap data
# export this data to visualize on datawrapper

merged = snap_df.join(census_crosswalk, left_on="GEOID", right_on="Tract", how="left")

merged.write_csv("data/clean_snap.csv")

In [6]:
# create a list of neighborhoods in the south side

south_side = ["Armour Square", "Bridgeport", "Douglas", "Englewood", 
              "Fuller Park", "Grand Boulevard", "Greater Grand Crossing",
              "Hyde Park", "Kenwood", "Oakland", "South Shore", 
              "Washington Park", "Woodlawn"] 


southwest = ["Archer Heights", "Brighton Park", "Chicago Lawn", "Clearing",
              "Gage Park", "Garfield Ridge", "McKinley Park", "New City",
              "West Elsdon", "West Englewood", "West Lawn"]	

far_southwest = ["Ashburn", "Auburn Gresham", "Beverly", "Morgan Park",
                 "Mount Greenwood", "Washington Heights"]

far_southeast = ["Avalon Park", "Burnside", "Calumet Heights", "Chatham",
                 "East Side", "Hegewisch", "Pullman", "Riverdale", "Roseland",
                 "South Chicago", "South Deering", "West Pullman"]

total_south = south_side + southwest + far_southwest + far_southeast

In [7]:
# estimate number of SNAP recipients by multipying the tract's population with 
# the tract's % on SNAP 

snap_full = merged.with_columns(
    (pl.col("Population") * pl.col("SNP_2019-2023")).alias("snap_recipients")
)

In [20]:
# pct of snap recipients in south side out of whole city

snap_southside = snap_full.filter(pl.col("CommunityAreaName").is_in(total_south)).select("snap_recipients").sum()

snap_chicago = snap_full.select("snap_recipients").sum()

pct_snap_southside = snap_southside / snap_chicago

print("South Side SNAP Population:", snap_southside)
print("Chicago SNAP Population:", snap_chicago)
print("South Side SNAP Pct:", pct_snap_southside)

South Side SNAP Population: shape: (1, 1)
┌─────────────────┐
│ snap_recipients │
│ ---             │
│ f64             │
╞═════════════════╡
│ 2.8263e7        │
└─────────────────┘
Chicago SNAP Population: shape: (1, 1)
┌─────────────────┐
│ snap_recipients │
│ ---             │
│ f64             │
╞═════════════════╡
│ 5.5406e7        │
└─────────────────┘
South Side SNAP Pct: shape: (1, 1)
┌─────────────────┐
│ snap_recipients │
│ ---             │
│ f64             │
╞═════════════════╡
│ 0.510108        │
└─────────────────┘


In [ ]:
# pct of south side residents in chicago's population

pop_southside = snap_full.filter(pl.col("CommunityAreaName").is_in(total_south)).select("Population").sum()

pop_chicago = snap_full.select("Population").sum()

pct_pop_southside = pop_southside / pop_chicago

print("South Side Population:", pop_southside)
print("Chicago Population:", pop_chicago)
print("South Side Pct:", pct_pop_southside)

South Side Population: shape: (1, 1)
┌────────────┐
│ Population │
│ ---        │
│ i64        │
╞════════════╡
│ 1013426    │
└────────────┘
Chicago Population: shape: (1, 1)
┌────────────┐
│ Population │
│ ---        │
│ i64        │
╞════════════╡
│ 2804697    │
└────────────┘
South Side Pct: shape: (1, 1)
┌────────────┐
│ Population │
│ ---        │
│ f64        │
╞════════════╡
│ 0.361332   │
└────────────┘
